In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline

In [ ]:
all_events=pd.read_csv('raw_data/tianchi_fresh_comp_train_user.csv')

In [ ]:
all_events['time']=pd.DatetimeIndex(all_events.time)

In [ ]:
all_items=pd.read_csv('raw_data/tianchi_fresh_comp_train_item.csv')
target_items=set(all_items.item_id)

In [ ]:
all_not_browsed_events=all_events[all_events.behavior_type>1]
all_not_browsed_events=all_not_browsed_events[all_not_browsed_events.item_id.apply(lambda id:id in target_items)]
len(all_not_browsed_events)

In [ ]:
all_not_browsed_events.time=pd.DatetimeIndex(all_not_browsed_events.time)
last_time=all_not_browsed_events.time.sort_values().iloc[-1]
last_day=pd.Timestamp(year=last_time.year,month=last_time.month,day=last_time.day)
last_day_not_br_evts=all_not_browsed_events[all_not_browsed_events.time>last_day]
len(last_day_not_br_evts)

In [ ]:
last_day_not_br_evts=last_day_not_br_evts[['user_id','item_id']]
pred=[]
for i in last_day_not_br_evts.index:
    uid=last_day_not_br_evts.loc[i,'user_id']
    iid=last_day_not_br_evts.loc[i,'item_id']
    if len(last_day_not_br_evts[np.logical_and(
        last_day_not_br_evts.user_id==uid,last_day_not_br_evts.item_id==iid)])==1:
        pred.append({'user_id':uid,'item_id':iid})
pred=pd.DataFrame(pred)

In [ ]:
pred[['user_id','item_id']].to_csv('output/last_day_not_br.csv',index=False)

In [ ]:
last_day_evts=all_events[all_events.time>last_day]
bought_items=last_day_evts[last_day_evts.behavior_type==4].item_id.unique()
all_br_evts=all_events[all_events.behavior_type==1]
last_br_evts=all_br_evts[all_br_evts.time>last_day]
last_br_evts=last_br_evts[last_br_evts.item_id.apply(lambda iid:iid in target_items and iid not in bought_items)]
len(last_br_evts)

In [ ]:
pred=[]
tmp=set(['a'])
for i in last_br_evts.index:
    uid=last_br_evts.loc[i,'user_id']
    iid=last_br_evts.loc[i,'item_id']
    k=str(uid)+'-'+str(iid)
    c=len(last_br_evts[np.logical_and(
        last_br_evts.user_id==uid,last_br_evts.item_id==iid)])
    if k not in tmp and c>1:
        pred.append({'user_id':uid,'item_id':iid,'cnt':c})
        tmp.add(k)
pred=pd.DataFrame(pred)
len(pred)

In [ ]:
pred[pred.cnt>0][['user_id','item_id']].to_csv('output/last_most_br_c1.csv',index=False)